Autor: Norton Martin Reichert Trennepohl 

# Questão 1

Nota: o algoritmo aqui mostrado funciona independentemente do número de lâminas ser par ou ímpar.

## Bibliotecas utilizadas:

In [96]:
import numpy as np
import math
import matplotlib.pyplot as plt

# Para mostrar apenas 3 casas decimais
np.set_printoptions(precision=3)

## Constantes e dados de entrada:

In [97]:
# Direções: informar aqui a direção de cada lâmina (em graus), de CIMA para BAIXO em relação ao laminado:
direc_deg = [45,0,0,45]

# Conversão para radianos
direc = np.radians(direc_deg)
n = len(direc)

#Vetor de carregamentos (elementos não-nulos estão em unidades de N/mm)
# Informar aqui os esforços, obedecendo a convenção:
#carreg = [[Nx],[Ny],[Nxy],[Mx],[My],[Mxy]]

carreg = [[1000*1000],[200*1000],[0],[0],[0],[0]] #N/m

#Espessura de cada lâmina (esp): informar 
esp = 3E-3 #m

#Espessura do laminado
h = n*esp #m

# Dados do material: informar aqui:
E11 = 19.76E9 #Pa
E22 = 1.97E9 #Pa
nu12 = 0.35
G12 = 0.7E9 #Pa

#Relação entre coeficientes de Poisson
nu21 = (E22*nu12)/E11

## Matriz de rigidez reduzida transformada no sistema global de coordenadas:

In [98]:
Q11 = E11/(1-nu12*nu21)
Q22 = E22/(1-nu12*nu21)
Q66 = G12
Q12 = (nu12*E22)/(1-nu12*nu21)
Q21 = Q12

Q = np.array([[Q11, Q12, 0],[Q21, Q22, 0], [0, 0, Q66]])
cos = np.cos(direc)
sin = np.sin(direc)

# Inicialização dos vetores:
T = [[0 for _ in range(1)] for _ in range(n)]
T_inv = [[0 for _ in range(1)] for _ in range(n)]
Q_dash = [[0 for _ in range(1)] for _ in range(n)]
Reuter = [[1,0,0],[0,1,0],[0,0,2]] # Matriz de Reuter

# Cálculo da matriz Q_dash para cada uma das lâminas 
for i in range(n):
    T[i] = np.array([[cos[i]**2, sin[i]**2, 2*sin[i]*cos[i]],[sin[i]**2, cos[i]**2, -2*sin[i]*cos[i]],[-sin[i]*cos[i], sin[i]*cos[i], cos[i]**2-sin[i]**2]])
    T_inv[i] = np.linalg.inv(T[i])
    Q_dash[i] = T_inv[i]@Q@Reuter@T[i]@np.linalg.inv(Reuter)
#   print(Q_dash[i])


## Matriz "ABBD":

In [99]:
# Matriz A: rigidez à tração e compressão
# Inicialização
A_local = [[0 for _ in range(1)] for _ in range(n)]
A_global = 0

for i in range(n):
    A_local[i] = Q_dash[i]*(((((n/2)-(i+1))/n)*-h) - (((n/2 - i)/n)*-h))
    #A_local[i] = Q_dash[i]*(h/n) # retorna os mesmos resultados, significa que está ok
    A_global = A_global + A_local[i]
#print(A_global)
    
# Matriz B: acoplamento entre rigidez no plano e rigidez à flexão
B_local = [[0 for _ in range(1)] for _ in range(n)]
B_global = 0
for i in range(n):
    B_local[i] = 0.5*Q_dash[i]*(((((n/2)-(i+1))/n)*-h)**2 - (((n/2 - i)/n)*-h)**2)
    B_global = B_global + B_local[i]
#print(B_global)

# Matriz D: rigidez à flexão ou torção
D_local = [[0 for _ in range(1)] for _ in range(n)]
D_global = 0
for i in range(n):
    D_local[i] = (1/3)*Q_dash[i]*(((((n/2)-(i+1))/n)*-h)**3 - (((n/2 - i)/n)*-h)**3)
    D_global = D_global + D_local[i]
#print(D_global)

# Combinando as matrizes em uma só   
linha1 = np.vstack((A_global,B_global))
linha2 = np.vstack((B_global,D_global))
ABBD = np.hstack((linha1,linha2))
print("Matriz ABBD:")
print(ABBD)

Matriz ABBD:
[[1.593e+08 3.508e+07 2.701e+07 1.455e-11 0.000e+00 0.000e+00]
 [3.508e+07 5.126e+07 2.701e+07 0.000e+00 0.000e+00 0.000e+00]
 [2.701e+07 2.701e+07 3.510e+07 0.000e+00 0.000e+00 0.000e+00]
 [1.455e-11 0.000e+00 0.000e+00 1.185e+03 6.613e+02 5.673e+02]
 [0.000e+00 0.000e+00 0.000e+00 6.613e+02 8.610e+02 5.673e+02]
 [0.000e+00 0.000e+00 0.000e+00 5.673e+02 5.673e+02 6.616e+02]]


Como era esperado, a matriz [B] é nula, já que o laminado é simétrico.

## Deformações no plano médio e curvatura em relação ao sistema global:

In [100]:
# Inicialização do vetor
def_curv = [[0],[0],[0],[0],[0],[0]]

def_curv = np.linalg.inv(ABBD)@carreg

epsilon_0_global = np.vstack((def_curv[0],def_curv[1],def_curv[2]))
K_global = np.vstack((def_curv[3],def_curv[4],def_curv[5]))

print(def_curv)


[[ 6.874e-03]
 [ 3.340e-03]
 [-7.861e-03]
 [-1.621e-16]
 [ 7.566e-17]
 [ 7.412e-17]]


Percebe-se que os valores de curvatura são muito baixos (podem ser considerados como nulos), o que já era esperado pois não é realizado momento sobre a estrutura. Isso indica que os resultados são condizentes.

## Tensões e deformações em cada lâmina:

In [101]:

# Tensões e deformações em cada lâmina (coordenadas z referenciadas no plano médio de cada lâmina)
sigma_global = [[0 for _ in range(1)] for _ in range(n)]
sigma_local = [[0 for _ in range(1)] for _ in range(n)]
z = [[0 for _ in range(1)] for _ in range(n)]
epsilon_global = [[0 for _ in range(1)] for _ in range(n)]
epsilon_local = [[0 for _ in range(1)] for _ in range(n)]
y = [[0 for _ in range(1)] for _ in range(n)]

for i in range(n):
    z[i] = 0.5*(((((n/2)-(i+1))/n)*-h) + (((n/2 - i)/n)*-h))
    sigma_global[i] = (Q_dash[i]@(epsilon_0_global + (z[i]*K_global)))*10**(-6)
    print("==============================")
    print("Resultados lâmina %d:" %(i+1))
    print("Tensão na lâmina no sistema global de coordenadas (MPa): ")
    print(sigma_global[i])
    sigma_local[i] = T[i]@sigma_global[i]
    print("Tensão na lâmina no sistema local de coordenadas (MPa):")
    print(sigma_local[i])
    epsilon_global[i] = epsilon_0_global + z[i]*K_global
    print("Deformação no plano médio da lâmina no sistema global de coordenadas:")
    print(epsilon_global[i])
    epsilon_local[i] = T[i]@epsilon_global[i]
    print("Deformação no plano médio da lâmina no sistema local de coordenadas:")
    print(epsilon_local[i])


    

Resultados lâmina 1:
Tensão na lâmina no sistema global de coordenadas (MPa): 
[[26.822]
 [21.874]
 [ 5.502]]
Tensão na lâmina no sistema local de coordenadas (MPa):
[[29.85 ]
 [18.845]
 [-2.474]]
Deformação no plano médio da lâmina no sistema global de coordenadas:
[[ 0.007]
 [ 0.003]
 [-0.008]]
Deformação no plano médio da lâmina no sistema local de coordenadas:
[[-0.003]
 [ 0.013]
 [-0.002]]
Resultados lâmina 2:
Tensão na lâmina no sistema global de coordenadas (MPa): 
[[139.845]
 [ 11.46 ]
 [ -5.502]]
Tensão na lâmina no sistema local de coordenadas (MPa):
[[139.845]
 [ 11.46 ]
 [ -5.502]]
Deformação no plano médio da lâmina no sistema global de coordenadas:
[[ 0.007]
 [ 0.003]
 [-0.008]]
Deformação no plano médio da lâmina no sistema local de coordenadas:
[[ 0.007]
 [ 0.003]
 [-0.008]]
Resultados lâmina 3:
Tensão na lâmina no sistema global de coordenadas (MPa): 
[[139.845]
 [ 11.46 ]
 [ -5.502]]
Tensão na lâmina no sistema local de coordenadas (MPa):
[[139.845]
 [ 11.46 ]
 [ -5.5

## Análise de falhas: Máxima tração

In [102]:
#tensoes_lim = [0,0,0,0,0] #inserir aqui os valores correspondentes
XT = 1
XC = 1
YT = 1
YC = 1
S12 = 1

CS_1 = [[0 for _ in range(1)] for _ in range(n)]
CS_2 = [[0 for _ in range(1)] for _ in range(n)]
CS_12 = [[0 for _ in range(1)] for _ in range(n)]


#sigma_local_transp = np.transpose(sigma_local)

for i in range(n):
    #print(i)
    if sigma_local[i][0,0] > 0:
        CS_1[i] = XT/sigma_local[i][0,0]
    else:
        CS_1[i] = -XC/sigma_local[i][0,0]


for i in range(n):
    if sigma_local[i][1,0] > 0:
        CS_2[i] = YT/sigma_local[i][1,0] 
    else:
        CS_2[i] = -YC/sigma_local[i][1,0] 

for i in range(n):
    if sigma_local[i][2,0] > 0:
        CS_12[i] = S12/abs(sigma_local[i][2,0])
    else:
        CS_12[i] = S12/abs(sigma_local[i][2,0])
   


#s_global = np.min()

# resultados:
print(CS_1)
print(CS_2)
print(CS_12)




[0.033500764855104954, 0.007150769741106406, 0.007150769741106406, 0.033500764855104954]
[0.053063725417847304, 0.08726319530532152, 0.08726319530532152, 0.053063725417847325]
[0.404217312081006, 0.18173889190828685, 0.18173889190828685, 0.40421731208100653]


## Tsai-Hill

In [103]:
FS_TH = [[0 for _ in range(1)] for _ in range(n)]
MS_TH = [[0 for _ in range(1)] for _ in range(n)]

for i in range(n):
    if sigma_local[i][0,0] > 0 and sigma_local[i][1,0] > 0:
        FS_TH[i] = np.sqrt((sigma_local[i][0,0]/XT)**2 + (sigma_local[i][1,0]/YT)**2 - (sigma_local[i][1,0]*sigma_local[i][0,0]/(XT**2)+(sigma_local[i][2,0]/S12)**2))
        MS_TH[i] = 1/FS_TH[i]
        #print(MS_TH)  
    elif sigma_local[i][0,0] < 0 and sigma_local[i][1,0] > 0:
        FS_TH[i] = np.sqrt((sigma_local[i][0,0]/XC)**2 + (sigma_local[i][1,0]/YT)**2 - (sigma_local[i][1,0]*sigma_local[i][0,0]/(XC**2)+(sigma_local[i][2,0]/S12)**2))
        MS_TH[i] = 1/FS_TH[i] 
    elif sigma_local[i][0,0] < 0 and sigma_local[i][1,0] < 0:
        FS_TH[i] = np.sqrt((sigma_local[i][0,0]/XC)**2 + (sigma_local[i][1,0]/YC)**2 - (sigma_local[i][1,0]*sigma_local[i][0,0]/(XC**2)+(sigma_local[i][2,0]/S12)**2))
        MS_TH[i] = 1/FS_TH[i]
    elif sigma_local[i][0,0] > 0 and sigma_local[i][1,0] < 0:
        FS_TH[i] = np.sqrt((sigma_local[i][0,0]/XT)**2 + (sigma_local[i][1,0]/YC)**2 - (sigma_local[i][1,0]*sigma_local[i][0,0]/(XT**2)+(sigma_local[i][2,0]/S12)**2))
        MS_TH[i] = 1/FS_TH[i]   


print(MS_TH)


[0.03841843458539936, 0.007442172445064681, 0.007442172445064681, 0.03841843458539936]


## Tsai-Wu

In [111]:
import math
F1 = 1/XT + 1/XC
F2 = 1/YT + 1/YC
F11 = -1/(XT*XC)
F22 = -1/(YT*YC)
F66 = (1/S12)**2
F12 = -0.5*np.sqrt(F11*F22)

A = [[0 for _ in range(1)] for _ in range(n)]
B = [[0 for _ in range(1)] for _ in range(n)]
Sf_pos = [[0 for _ in range(1)] for _ in range(n)]
Sf_neg = [[0 for _ in range(1)] for _ in range(n)]
Sf = [[0 for _ in range(1)] for _ in range(n)]

for i in range(n):
    A[i] = F11*sigma_local[i][0,0]**2 + F22*sigma_local[i][1,0]**2 + F66*sigma_local[i][2,0]**2 + 2*F12*sigma_local[i][0,0]*sigma_local[i][1,0]
    B[i] = F1*sigma_local[i][0,0] + F2*sigma_local[i][1,0]
    Sf_pos[i] = (-B[i] + np.sqrt(B[i]**2 + 4*A[i]))/(2*A[i])
    Sf_neg[i] = abs((-B[i] - np.sqrt(B[i]**2 + 4*A[i]))/(2*A[i]))

    Sf[i] = min(Sf_pos[i],Sf_neg[i])

print(Sf)

[0.013785161374900502, 0.005216127039755946, 0.005216127039755946, 0.013785161374900502]
